In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
config_file = "configs/vgg_ssd300_voc0712_tdt4265_server.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-03-23 16:34:35,416 SSD INFO: Loaded configuration file configs/vgg_ssd300_voc0712_tdt4265_server.yaml
2021-03-23 16:34:35,418 SSD INFO: 
MODEL:
    NUM_CLASSES: 21
INPUT:
    IMAGE_SIZE: [300, 300]
    PIXEL_STD: [0.003921569, 0.003921569, 0.003921569] # Set pixel std to 1/255 as VGG is pre-trained on images in range (roughly) [-127.5, 127.5]
DATASETS:
    TRAIN: ("voc_2007_trainval", "voc_2012_trainval")
    TEST: ("voc_2007_test", )
SOLVER:
    MAX_ITER: 5000
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 5e-4

OUTPUT_DIR: 'outputs/vgg_VOC'
EVAL_STEP: 2500
MODEL_SAVE_STEP: 2500
DATASET_DIR: "/work/datasets/"

2021-03-23 16:34:35,419 SSD INFO: Running with config:
DATASETS:
  TEST: ('voc_2007_test',)
  TRAIN: ('voc_2007_trainval', 'voc_2012_trainval')
DATASET_DIR: /work/datasets/
DATA_LOADER:
  NUM_WORKERS: 4
  PIN_MEMORY: True
EVAL_STEP: 2500
INPUT:
  IMAGE_SIZE: [300, 300]
  PIXEL_MEAN: [123.675, 116.28, 103.53]
  PIXEL_STD: [0.003921569, 0.003921569, 0.003921569]
LOG_STEP: 10
MODEL

In [4]:
model = start_train(cfg)

Downloading: "https://s3.amazonaws.com/amdegroot-models/vgg16_reducedfc.pth" to /home/fredralm/.torch/models/vgg16_reducedfc.pth



Detector initialized. Total Number of params:  26.29M
Backbone number of parameters: 22.94M
SSD Head number of parameters: 3.34M
2021-03-23 16:34:43,862 SSD.trainer INFO: Loading checkpoint from outputs/vgg_VOC/model_002500.pth
2021-03-23 16:34:44,044 SSD.trainer INFO: Loading optimizer from outputs/vgg_VOC/model_002500.pth
2021-03-23 16:34:44,159 SSD.trainer INFO: Start training ...
2021-03-23 16:35:02,063 SSD.trainer INFO: iter: 002510, lr: 0.00050, total_loss: 4.867 (4.867), reg_loss: 1.382 (1.382), cls_loss: 3.485 (3.485), time: 1.652 (1.652), eta: 1:08:33, mem: 6718M
2021-03-23 16:35:08,924 SSD.trainer INFO: iter: 002520, lr: 0.00050, total_loss: 4.947 (4.907), reg_loss: 1.470 (1.426), cls_loss: 3.477 (3.481), time: 0.686 (1.169), eta: 0:48:19, mem: 6718M
2021-03-23 16:35:15,787 SSD.trainer INFO: iter: 002530, lr: 0.00050, total_loss: 4.954 (4.923), reg_loss: 1.400 (1.418), cls_loss: 3.554 (3.505), time: 0.686 (1.008), eta: 0:41:30, mem: 6718M
2021-03-23 16:35:22,661 SSD.trainer 

100%|██████████| 496/496 [02:19<00:00,  3.55it/s]


2021-03-23 17:06:27,600 SSD.inference INFO: mAP: 0.4700
aeroplane       : 0.5392
bicycle         : 0.5576
bird            : 0.4601
boat            : 0.2165
bottle          : 0.1636
bus             : 0.5319
car             : 0.7160
cat             : 0.6447
chair           : 0.3009
cow             : 0.4186
diningtable     : 0.3710
dog             : 0.5570
horse           : 0.5508
motorbike       : 0.5485
person          : 0.6032
pottedplant     : 0.1872
sheep           : 0.4319
sofa            : 0.4440
train           : 0.5988
tvmonitor       : 0.5584

2021-03-23 17:06:27,693 SSD.trainer INFO: Saving checkpoint to outputs/vgg_VOC/model_final.pth
2021-03-23 17:06:27,997 SSD.trainer INFO: Total training time: 0:31:42 (0.3804 s / it)


In [5]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)

2021-03-23 15:00:19,172 SSD INFO: Start evaluating...
2021-03-23 15:00:19,326 SSD.inference INFO: Evaluating mnist_detection_val dataset(1000 images):


100%|██████████| 100/100 [00:12<00:00,  7.69it/s]


2021-03-23 15:00:33,086 SSD.inference INFO: mAP: 0.8753
0               : 0.8940
1               : 0.8095
2               : 0.8760
3               : 0.8971
4               : 0.8807
5               : 0.8830
6               : 0.8859
7               : 0.8569
8               : 0.8900
9               : 0.8794



[{'metrics': {'mAP': 0.8752565431542013,
   '0': 0.8939699650469805,
   '1': 0.8094709526574602,
   '2': 0.8760044641851805,
   '3': 0.897146832405082,
   '4': 0.8807326189016644,
   '5': 0.883031870897962,
   '6': 0.8859388314999235,
   '7': 0.8569403048356649,
   '8': 0.8899598842157962,
   '9': 0.8793697068962991}}]